In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set model storage path inside Google Drive
mistral_path = "/content/drive/MyDrive/mistral_models/7B-Instruct-v0.3"

# Create the directory if it doesn't exist
os.makedirs(mistral_path, exist_ok=True)

print(f"Model directory set: {mistral_path}")

To Download the model on your drive

In [ ]:
# from huggingface_hub import snapshot_download

# # Download the model to Google Drive (persists after reset)
# snapshot_download(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#     local_dir=mistral_path,
#     allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"]
# )

# print("Model downloaded successfully!")


In [ ]:
# import shutil

# # Rename model file
# old_model_path = os.path.join(mistral_path, "consolidated.safetensors")
# new_model_path = os.path.join(mistral_path, "model.safetensors")

# if os.path.exists(old_model_path):
#     shutil.move(old_model_path, new_model_path)
#     print("Model file renamed!")

# # Rename tokenizer file
# old_tokenizer_path = os.path.join(mistral_path, "tokenizer.model.v3")
# new_tokenizer_path = os.path.join(mistral_path, "tokenizer.model")

# if os.path.exists(old_tokenizer_path):
#     shutil.move(old_tokenizer_path, new_tokenizer_path)
#     print("Tokenizer file renamed!")

###FP16 (Half Precision, 2 bytes per parameter) takes lot of ram so i am sifting to int4

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# mistral_path = "mistralai/Mistral-7B-Instruct-v0.3"

# # Configure 4-bit quantization
# bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(mistral_path, use_fast=False)

# # Load model with quantization
# model = AutoModelForCausalLM.from_pretrained(
#     mistral_path,
#     quantization_config=bnb_config,  # Enables 4-bit quantization
#     device_map="auto",  # Automatically assigns model to available GPU(s)
# )

# print("Model and Tokenizer Loaded Successfully (4-bit Quantized)!")

In [ ]:
# Move model to correct device
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # Example input
# input_text = "write an eassay on What is the meaning of life?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


In [ ]:
# input_text = "i am kalam write an eassay 100 words"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


In [ ]:
# input_text = "i am kalam write an eassay 100 words continue from Education is a powerful tool for social and economic mobility, enabling individuals to access"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


###shift to 8bit quantization

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# mistral_path = "mistralai/Mistral-7B-Instruct-v0.3"

# # Configure 8-bit quantization
# bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(mistral_path, use_fast=False)

# # Load model with quantization
# model = AutoModelForCausalLM.from_pretrained(
#     mistral_path,
#     quantization_config=bnb_config,  # Enables 8-bit quantization
#     device_map="auto",  # Automatically assigns model to available GPU(s)
# )

# print("Model and Tokenizer Loaded Successfully (8-bit Quantized)!")


In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Example input
# input_text = "What is the meaning of life?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# # Generate response
# output = model.generate(input_ids, max_length=100)
# response = tokenizer.decode(output[0], skip_special_tokens=True)

# print("\nGenerated Response:")
# print(response)


In [ ]:
!pip install mistral_inference transformers sentencepiece torch safetensors accelerate transformers trl torch peft wandb bitsandbytes


In [ ]:
#imports
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification
from trl import PPOTrainer, PPOConfig
import pandas as pd
import numpy as np


ModuleNotFoundError: No module named 'trl'

In [ ]:
#dataset
df = pd.read_json('/content/drive/MyDrive/PreProcessedDataset/train.json',lines = True)

In [ ]:
# df.head()
dataset=df["source"]
dataset.head()

In [ ]:
class ReinforcementModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # LLM model
        self.mistral_path = "/content/drive/MyDrive/mistral_models/7B-Instruct-v0.3"

        # Reward model
        self.reward_model_name = "OpenAssistant/reward-model"

        # 8-bit quantization config
        bnb_config = BitsAndBytesConfig(load_in_8bit=True)

        # Pretrained tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.mistral_path, use_fast=False)

        # Load pretrained model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.mistral_path,
            quantization_config=bnb_config,
            device_map="auto"
        )

        # Device
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load reward model
        self.reward_model = self._load_reward_model()

        # PPO config
        ppo_config = PPOConfig(
            model_name=self.mistral_path,
            learning_rate=1.41e-5,
            batch_size=4,
            mini_batch_size=2,
            gradient_accumulation_steps=8
        )

        # Initialize PPO Trainer
        self.ppo_trainer = PPOTrainer(
            model=self.model,
            config=ppo_config,
            dataset=df,
            tokenizer=self.tokenizer,
            reward_model=self.reward_model
        )

    def _load_reward_model(self):
        """ Load the reward model """
        reward_model = AutoModelForSequenceClassification.from_pretrained(
            self.reward_model_name,
            device_map="auto"
        )
        return reward_model

    def generate_text(self, prompt):
        """ Generate text using the model """
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.device)
        output = self.model.generate(input_ids, max_length=100)#write now its 100 (as it would require high computation) and GPU
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return response

    def train(self, epochs=3):
        """ Train the model using PPO """
        for epoch in range(epochs):
            for dp in self.ppo_trainer.dataloader:
                query = dp["source"]  # Title/Prompt
                response = self.generate_text(query)

                # Compute reward (Using reward model)
                inputs = self.tokenizer(response, return_tensors="pt", padding=True, truncation=True).to(self.device)
                rewards = self.reward_model(**inputs).logits.squeeze()

                # Train PPO step
                self.ppo_trainer.step([query], [response], [rewards])

            print(f"Epoch {epoch+1}/{epochs} completed!")

# Initialize Model
rm = ReinforcementModel()

# Train Model
rm.train(epochs=3)
